## Description
In this notebook, I am improving the result I got earlier as a base solution 2440 in my other notebook, the data is stored in a public dataset.

## Sources
- https://www.kaggle.com/renokan/kaggle-santa-2021-create-solution-2440
- https://www.kaggle.com/renokan/dataset-kaggle-santa-2021 "Dataset Kaggle Santa-2021"

# 1. Import & Load & Extract

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("max_colwidth", 80)

import copy

In [ ]:
def encode_symbols(data, symbols):
    """ \o/ """
    if isinstance(data, pd.Series) and isinstance(symbols, dict):

        data = data.copy()

        for key, value in symbols.items():
            data = data.str.replace(value, str(key), regex=True)    

    return data


def decode_symbols(data, symbols):
    """ \o/ """
    if isinstance(data, pd.Series) and isinstance(symbols, dict):

        data = data.copy()

        for key, value in symbols.items():
            data = data.str.replace(str(key), value, regex=True)    

    return data


def create_data(df, symbols=None):
    """ \o/ """
    lines_col = "line"
    lines_list = df[lines_col].unique()
    other_cols = [col for col in df.columns if col != lines_col]
    
    data_dict = {}
    for x_line in lines_list:
        x_mask = df[lines_col] == x_line
        x_data = df.loc[x_mask, other_cols].copy().reset_index(drop=True)
        
        if symbols:
            for col in x_data.columns:
                x_data[col] = encode_symbols(x_data[col], symbols)
        
        x_data['result'] = ""
        data_dict[x_line] = x_data
    
    return data_dict


def extract_combs(part_line, all_combs):
    """ \o/ """
    if not isinstance(part_line, str) or not isinstance(all_combs, pd.Series):
        return False
    
    result = [comb for comb in all_combs if comb in part_line]
    
    return pd.Series(result)


def get_base_result(combinations, symbols):
    """ \o/ """
    base_result = {}    
    data_dict = create_data(combinations, symbols)
    
    for x_line in data_dict.keys():
        x_data = data_dict.get(x_line)
        base_result[x_line] = "12" + x_data['body'] + x_data['tail'].str.slice(2)
    
    return base_result


def get_splitted_combs(combinations):
    """ \o/ """
    result = []
    split_list = [(0, 7), (2, 15),
                  (10, 23), (18, 31), (26, 39),
                  (34, 47), (43, 56), (54, 62)
    ]
    
    for x_combs in combinations.values:
        x_result = []
        
        for split_steps in split_list:            
            i_start, i_stop = split_steps
            
            x_result.append(x_combs[i_start:i_stop])
        
        result.append(x_result)
        
    splitted_combs = pd.DataFrame(result)
    
    return splitted_combs


def _print_indents(combinations):
    """ \o/ """
    indents_list = [0, 2, 10, 18, 26, 34, 43, 54]
    
    if isinstance(combinations, pd.Series):
        for i, value in combinations.items():
            if i < len(indents_list):
                indent_value = indents_list[i]
            else:
                indent_value = 0

            print("{} {}".format(" " * indent_value, value))


def show_splitted_combs(combinations):
    """ \o/ """
    if isinstance(combinations, pd.Series):
        _print_indents(combinations)

    if isinstance(combinations, pd.DataFrame):
        for i in combinations.index:
            _print_indents(combinations.loc[i])

        print()
            

In [ ]:
path_to_permutations = "../input/santa-2021/permutations.csv"
all_permutations = pd.read_csv(path_to_permutations)

path_to_combinations = "../input/dataset-kaggle-santa-2021/combinations.csv"

combinations = pd.read_csv(path_to_combinations)
combinations.head()

In [ ]:
symbols = {
    0: '🌟',  # wildcard
    1: '🎅',  # first start combination
    2: '🤶',  # second start combination
    3: '🦌',
    4: '🧝',
    5: '🎄',
    6: '🎁',
    7: '🎀',
}

# 2. Solutions

In [ ]:
def get_result(data_dict, col_name=None):
    """ \o/ """
    result = {}
    
    if not col_name:
        col_name = "result"
    
    for x_key, x_value in data_dict.items():
        if isinstance(x_value, pd.DataFrame):
            x_value = x_value.get(col_name)
            
        result[x_key] = x_value.sum()
        
    return pd.Series(result, name='schedule')

## 2.1 Base solution 2440

In [ ]:
data_dict = create_data(combinations, symbols)
data_dict.keys()

In [ ]:
data_dict.get(1).head()

In [ ]:
for x_line in data_dict.keys():
    x_data = data_dict.get(x_line)
    x_data['result'] = "12" + x_data['body'] + x_data['tail'].str.slice(2)

In [ ]:
data_dict.get(1).head()

In [ ]:
result_2440 = get_result(data_dict)
result_2440

In [ ]:
result_2440.str.len()

## 2.2 Improved solution - 2438

In [ ]:
base_data_dict = get_base_result(combinations, symbols)
base_data_dict.keys()

In [ ]:
base_data_dict.get(1).head()

In [ ]:
position_list = [(0, 1), (2, 3)]

for x_line in base_data_dict.keys():
    for x, y in position_list:
        cutted_value = base_data_dict.get(x_line).pop(x)
        replaced_value = base_data_dict.get(x_line)[y]
        
        new_value = replaced_value[:-1] + "0" + cutted_value[1:]
        base_data_dict.get(x_line)[y] = new_value

In [ ]:
base_data_dict.get(1).head()

In [ ]:
result_2438 = get_result(base_data_dict)
result_2438

In [ ]:
result_2438.str.len()

## 2.3 Improved solution - 2435

In [ ]:
base_data_dict = get_base_result(combinations, symbols)
base_data_dict.keys()

In [ ]:
base_data_dict.get(1).head()

In [ ]:
data_replace = base_data_dict.get(1)
old_value = "1234567213456723145672341567234516723456172345612734561234657"

"""
1234567
  3456721345672
          4567231456723
                  5672341567234
                          6723451672345
                                  7234561723456
                                           3456127345612 
                                                      1234657


2     1263457216345726134572631457263415726345172634512763451235647
26    1257436215743625174362571436257413625743162574312657431267345


                                   
4567231456723
        5672341567234
                               *            *
                6723451672345
                        7234561723456
                               1234567
                                 3456721345672
                                        3456127345612
                                                   1234657


"""
del_value = (26, )
new_value = (0, "12574362157436251743625714362574136257431625743126574312673" + \
                "4567231456723415672345167234561023456721345602734561234657")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
data_replace = base_data_dict.get(2)
old_value = "1273456217345627134562731456273415627345162734512673451234576"

"""
1273456
  7345621734562
          3456271345627
                  4562731456273
                          5627341562734
                                  6273451627345
                                           7345126734512
                                                      1234576



3456271345627
        4562731456273
                5627341562734  *            *
                        6273451627345
                               1273456
                                 7345621734562
                                        7345126734512
                                                   1234576

"""
del_value = (24, )
new_value = (0, "12547362154736251473625417362547136254731625473126547312567" + \
                "3456271345627314562734156273451027345621734502673451234576")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
base_data_dict.get(3).head()

In [ ]:
data_replace = base_data_dict.get(3)
old_value = "1267345216734526173452671345267314526734152673412567341234567"

"""

1267345
  6734521673452
          7345261734526
                  3452671345267
                          4526731452673
                                  5267341526734
                                           6734125673412
                                                      1234567


7345261734526
        3452671345267
                4526731452673  *            *
                        5267341526734
                               1267345
                                 6734521673452
                                        6734125673412
                                                   1234567

"""
del_value = (23, )
new_value = (0, "12736542173654271365427316542736154273651427365124736512645" + \
                "7345261734526713452673145267341026734521673402567341234567")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
result_2435 = get_result(base_data_dict)
result_2435

In [ ]:
result_2435.str.len()

## 2.4 Improved solution - 2434

In [ ]:
base_data_dict = get_base_result(combinations, symbols)
base_data_dict.keys()

In [ ]:
data_replace = base_data_dict.get(1)
old_value = "1263457216345726134572631457263415726345172634512763451235647"

"""
3457261345726
        4572631457263          *            *
                5726341572634
                        7263451726345
                               1263457
                                 6345721634572
                                        6345127634512
                                                   1235647


"""
del_value = (26, )
new_value = (2, "1257436215743625174362571436257413625743162574312657431267" + \
                "3457261345726314572634157263451026345721634502763451235647")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
data_replace = base_data_dict.get(2)
old_value = "1234657213465723146572341657234615723465172346512734651235746"

"""

1234657
  3465721346572
          4657231465723
                  6572341657234
                          5723461572346
                                  7234651723465
                                           3465127346512
                                                      1235746


4657231465723
        6572341657234          *            *
                5723461572346
                        7234651723465
                               1234657
                                 3465721346572
                                        3465127346512
                                                   1235746

"""
del_value = (6, )
new_value = (3, "1246753214675324167532461753246715324675132467512346751237" + \
                "4657231465723416572346157234651023465721346502734651235746")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
base_data_dict.get(3).head()

In [ ]:
data_replace = base_data_dict.get(3)
old_value = "1267345216734526173452671345267314526734152673412567341234567"

"""

1267345
  6734521673452
          7345261734526
                  3452671345267
                          4526731452673
                                  5267341526734
                                           6734125673412
                                                      1234567


7345261734526
        3452671345267
                4526731452673  *            *
                        5267341526734
                               1267345
                                 6734521673452
                                        6734125673412
                                                   1234567

"""
del_value = (27, )
new_value = (0, "1274536217453627145362741536274513627453162745312674531265" + \
                "7345261734526713452673145267341026734521673402567341234567")

data_replace[new_value[0]] = new_value[1] 
_ = data_replace.pop(del_value[0])

In [ ]:
result_2434 = get_result(base_data_dict)
result_2434

In [ ]:
result_2434.str.len()

## 2.5 Improved solution - 2433

In [ ]:
base_data_dict = get_base_result(combinations, symbols)
base_data_dict.keys()

In [ ]:
base_data_dict.get(1).head()

In [ ]:
# === 1 ===
data_replace = base_data_dict.get(1)


# - 25 -
#        *
#        1236574
# 4236571423657
#          3657421365742
#                  6574231657423
#                          5742361574236   *
#                                  7423651742365
#                                          1236547    < 4 > 1265473
#                                                71243657
# 
# 4236571023657421365742316574236157423651702365471243657
# 
# 3657124365712
# 3657124
#  6571243
#   5712436
# ** 7124365
# Line 3
# 9  1243657
# 28 1243675

data_replace[25] = data_replace.pop(5) + \
	            "236571023657421365742316574236157423651702365471243657"

data_replace[4] = data_replace[4][:-7] + "1265473"

# === 2 ===
data_replace = base_data_dict.get(2)

# [...]3647 + 1253467[...] = 3647125364712 (Line 3)
data_replace[18] = data_replace.pop(16) + "1253674" + data_replace.pop(17)[:-7] + data_replace.pop(18)

data_replace[1] = data_replace.pop(15) + \
	            "245671024567321456732415673245167324561702456371234567"

data_replace[8] = data_replace[8][:-7] + "1234756"

# === 3 ===
data_replace = base_data_dict.get(3)

# [...]3657 + 1243675[...] = 3657124365712 (Line 1)
data_replace[9] = data_replace[9] + data_replace.pop(28)

# [...]4567 + 1234576[...] = 4567123456712 (Line 2)
data_replace[0] = data_replace[0] + data_replace.pop(1)

data_replace[21] = data_replace.pop(0) + \
	            "236471023647521364752316475236147523641702364571253647"

data_replace[4] = data_replace[4][:-7] + "1256743"

In [ ]:
result_2433 = get_result(base_data_dict)
result_2433

In [ ]:
result_2433.str.len()

## 2.6 Improved solution - 2430

In [ ]:
base_data_dict = get_base_result(combinations, symbols)
base_data_dict.keys()

In [ ]:
base_data_dict.get(1).head()

In [ ]:
"""
=== Line 1 ===
- 0 -
[...]6 < 6, 8, 13, 16, 32, 34, 36
61273456
    3456721345672
            4567231456723
                    5672341567234
                            6723451672345  *
                                    7234561723456
                                           1234567
                                                  1234657

 12734567213456723145672341567234516723456102345671234657

- 7 -
[...]7 < 0, 2, 4, 11, 24
71256347
    6347521634752
            3475261347526
                    4752631475263
                            7526341752634  *
                                    5263471526347
                                           1263475
                                                  1237654

 12563475216347526134752631475263417526347102634751237654

=== Line 2 ===
- 1 -
[...]7 < 8, 16, 20, 23, 28, 30, 32, 34
71234567
    4567321456732
            5673241567324
                    6732451673245
                            7324561732456  *
                                    3245671324567
                                           1245673
                                                  1234756

 12345673214567324156732451673245617324567102456731234756

- 11 -
71235467
    5467321546732
            4673251467325
                    6732541673254
                            7325461732546  *
                                    3254671325467
                                           1254673
                                                  1246375

 12354673215467325146732541673254617325467102546731246375

Line 3 > 8 : 5347126534712
34  [...]1265347 +
32  1265374[...]

Line 3 > 10 : 6735124673512
12 1246735
6  1246753

=== Line 3 ===

- 8 -
[...]7 < 0, 2, 4, 8, 9, 11, 14, 16, 17, 20, 26
71265347
    5347621534762
            3476251347625
                    4762531476253
                            7625341762534  *
                                    6253471625347
                                           1253476
                                                  1243567

 12653476215347625134762531476253417625347102534761243567

- 10 - 
[...]5 < 1, 22, 24, 31, 36, 38
51246735
    6735421673542
            7354261735426
                    3542671354267
                            5426731542673  *
                                    4267351426735
                                           1267354
                                                  1243756

 12467354216735426173542671354267315426735102673541243756

Line 1 > 0 : 3456127345612
29 [...]1273456 + 
3  1273465[...]

Line 1 > 7 : 6347125634712
20 [...]1256347 +
30 1256374[...]

Line 2 > 1 : 4567123456712
0  [...]1234567 +
1  1234576[...]

Line 2 > 11 : 5467123546712
2  [...]1235467 +
11 1235476[...]

"""

# === 1 ===
data_replace = base_data_dict.get(1)

_ = data_replace.pop(0)
data_replace[8] = data_replace[8] + \
                    "12734567213456723145672341567234516723456102345671234657"

_ = data_replace.pop(7)
data_replace[2] = data_replace[2] + \
                    "12563475216347526134752631475263417526347102634751237654"

# === 2 ===
data_replace = base_data_dict.get(2)

_ = data_replace.pop(1)
data_replace[8] = data_replace[8] + \
                    "12345673214567324156732451673245617324567102456731234756"

_ = data_replace.pop(11)
data_replace[16] = data_replace[16] + \
                    "12354673215467325146732541673254617325467102546731246375"

data_replace[34] = data_replace[34] + data_replace.pop(32)
data_replace[12] = data_replace[12] + data_replace.pop(6)

# === 3 ===
data_replace = base_data_dict.get(3)

_ = data_replace.pop(8)
data_replace[4] = data_replace[4] + \
                    "12653476215347625134762531476253417625347102534761243567"

_ = data_replace.pop(10)
data_replace[22] = data_replace[22] + \
                    "12467354216735426173542671354267315426735102673541243756"

data_replace[29] = data_replace[29] + data_replace.pop(3)
data_replace[20] = data_replace[20] + data_replace.pop(30)
data_replace[0] = data_replace[0] + data_replace.pop(1)
data_replace[2] = data_replace[2] + data_replace.pop(11)

In [ ]:
result_2430 = get_result(base_data_dict)
result_2430

In [ ]:
result_2430.str.len()

# 3. Save submission

In [ ]:
selected_result = result_2430
selected_result.str.len()

In [ ]:
save_submission = True

if save_submission:
    submission = selected_result.copy()
    
    submission = decode_symbols(submission, symbols)
    
    submission.to_csv('submission.csv', index=False)
    
print(save_submission)